In [ ]:
# Import libraries. You may or may not use all of these.
!pip install -q git+https://github.com/tensorflow/docs
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

try:
  # %tensorflow_version only exists in Colab.
  %tensorflow_version 2.x
except Exception:
  pass
import tensorflow as tf

from tensorflow import keras
from tensorflow.keras import layers

import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling

In [ ]:
# Import data
!wget https://cdn.freecodecamp.org/project-data/health-costs/insurance.csv
dataset = pd.read_csv('insurance.csv')
dataset.tail()

In [ ]:
# Split our data to features and labels & training and test
i =int(len(dataset) *0.8)
train_features = dataset[:i]
test_features = dataset[i:]
train_labels = train_features.pop('expenses')
test_labels = test_features.pop('expenses')
print(train_features.shape)
print(train_labels.shape)
print(test_features.shape)
print(test_labels.shape)

In [ ]:
train_features['age'].hist();

In [ ]:
train_features.sex.value_counts().plot(kind='barh');

In [ ]:
train_features['region'].value_counts().plot(kind='barh');

In [ ]:
train_features.smoker.value_counts().plot(kind='barh');

In [ ]:
pd.concat([train_features, train_labels], axis=1).groupby('smoker').expenses.mean().plot(kind='barh').set_xlabel('MEAN HEALTH COST INSURANCE FOR SMOKERS AND FOR NOT');

Handle categorical data

In [ ]:
CATEGORICAL_COLUMNS = ['sex', 'region', 'smoker']
train_features['sex'].replace({'male':0,'female':1},inplace=True)
test_features['sex'].replace({'male':0,'female':1},inplace=True)

train_features['smoker'].replace({'yes':1,'no':0},inplace=True)
test_features['smoker'].replace({'yes':1,'no':0},inplace=True)

train_features['region'].replace({'northwest':0,'southwest':1,'northeast':2,'southeast':3},inplace=True)
test_features['region'].replace({'northwest':0,'southwest':1,'northeast':2,'southeast':3},inplace=True)

Build model

In [ ]:
model = tf.keras.models.Sequential([
    tf.keras.layers.Input(shape=(len(train_features.keys()),)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(1)
])
model.compile(
    optimizer= tf.keras.optimizers.RMSprop(0.05),
    loss='mse',
    metrics=['mae', 'mse']
)
model.summary()

In [ ]:
# taken from tfdocs.modeling.EpochDots
class EpochDots(keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs):
    if epoch % 100 == 0:
      print()
      print('Epoch: {:d}, '.format(epoch), end='')
      for name, value in sorted(logs.items()):
        print('{}:{:0.4f}'.format(name, value), end=', ')
      print()

    print('.', end='')

In [ ]:
r = model.fit(train_features, train_labels, epochs=10,
              verbose=1, callbacks=[EpochDots()])

In [ ]:
test_dataset = test_features

In [ ]:
res = model.evaluate(test_dataset, test_labels, verbose=2)
print(res)

Challenge

In [ ]:
# RUN THIS CELL TO TEST YOUR MODEL. DO NOT MODIFY CONTENTS.
# Test model by checking how well the model generalizes using the test set.
loss, mae, mse = model.evaluate(test_dataset, test_labels, verbose=2)

print("Testing set Mean Abs Error: {:5.2f} expenses".format(mae))

if mae < 3500:
  print("You passed the challenge. Great job!")
else:
  print("The Mean Abs Error must be less than 3500. Keep trying.")

# Plot predictions.
test_predictions = model.predict(test_dataset).flatten()

a = plt.axes(aspect='equal')
plt.scatter(test_labels, test_predictions)
plt.xlabel('True values (expenses)')
plt.ylabel('Predictions (expenses)')
lims = [0, 50000]
plt.xlim(lims)
plt.ylim(lims)
_ = plt.plot(lims,lims)
